# Scrapping Master Degrees

### Objetivo

Com este projeto pretende-se desenvolver uma plataforma para inspecionar, a nível nacional, a oferta de mestrados e a compatibilidade dos mesmos com as necessidades atuais das empresas.

#### Benefícios

Esta plataforma pretende fornecer uma ferramenta analítica capaz de analisar a oferta formativa e compará-la com a oferta profissional, de uma forma rápida e apelativa.

Import packages

In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
# from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as BraveService
from webdriver_manager.core.os_manager import ChromeType
from selenium.webdriver.chrome.service import Service
import time
from bs4 import BeautifulSoup
import pandas as pd
import os
from lxml import etree
from dotenv import load_dotenv
load_dotenv("variables.env")

True

In [2]:

# # element = WebDriverWait(browser, 20).until(EC.visibility_of_element_located((By.XPATH, )))
# #accessing Chromedriver
# option = Options()
# option.add_argument("--disable-infobars")
# option.add_argument("start-maximized")
# option.add_argument("--disable-extensions")
# option.add_argument("--incognito")
    
#     # Pass the argument 1 to allow and 2 to block
# option.add_experimental_option("prefs", {
#      "profile.default_content_setting_values.notifications": 1
# })
#     # chromedriver should be in the same folder as file
# service = Service(executable_path="./chromedriver-win64/chromedriver.exe")
# browser = webdriver.Chrome(service=service, options=option)

# browser.get("https://sapui5.hana.ondemand.com/")
# browser.implicitly_wait(5)
# time.sleep(3)
# browser.find_element(By.XPATH, value="/html[1]/body[1]/div[1]/div[1]/footer[1]/div[1]/button[1]").click()
# time.sleep(3)
# browser.find_element(By.XPATH, value="/html[1]/body[1]/div[3]/section[1]/div[1]/div[1]/div[1]/div[1]/div[1]/header[1]/div[2]/div[1]/div[1]/div[1]/div[4]/div[1]").click()
# # print(browser.page_source)
# soup = BeautifulSoup(browser.page_source, 'html.parser')
# print(soup.prettify)
# time.sleep(500)



In [4]:
brave_user_datra_dir=r"C://Users//Bernardo//AppData//Local//BraveSoftware//Brave-Browser//User Data//Default"
#Options
options = webdriver.ChromeOptions()
options.add_argument("--disable-infobars")
options.add_argument("start-minimized")
options.add_argument("--disable-extensions")
options.add_argument("--incognito")
options.binary_location = os.environ.get('BINARY_LOCATION')

driver_path = './chromedriver-win64/chromedriver.exe'
service = Service(driver_path)
browser = webdriver.Chrome(service=service, options=options)
browser.get("https://www.dges.gov.pt/simges/public/www/cursos_instituicoes")
browser.implicitly_wait(5)
browser.find_element(By.XPATH, value="//body/div[4]/div[1]/div[1]/div[1]/div[1]/form[1]/div[1]/div[2]/div[2]").click()
browser.implicitly_wait(5)
browser.find_element(By.XPATH, value="/html[1]/body[1]/div[4]/div[1]/div[1]/div[1]/div[1]/form[1]/div[1]/button[1]").click()
time.sleep(5)
buttons=["/html[1]/body[1]/div[4]/div[1]/div[1]/div[1]/div[1]/div[2]/div[1]/ul[1]/li[1]/a[1]","/html[1]/body[1]/div[4]/div[1]/div[1]/div[1]/div[1]/div[2]/div[1]/ul[1]/li[2]/a[1]","/html[1]/body[1]/div[4]/div[1]/div[1]/div[1]/div[1]/div[2]/div[1]/ul[1]/li[3]/a[1]"]
Unis = {"Universidade":[],"Link_DGES":[],"Nome":[],"Estado da IES":[],"Unidade Organica":[],"Subsistema":[],"Natureza":[],"Tipo de ensino":[],"Link_Website":[],"Morada":[],"Código Postal":[],"Telefone":[],"Email":[],"Distrito":[]}
for button in buttons:
    browser.find_element(By.XPATH, value=button).click()
    time.sleep(1)
    html = browser.page_source
    soup = BeautifulSoup(html, 'html.parser')
    # Find all <td> elements
    td_elements = soup.find_all('td')
    for td in td_elements:
        # Find all <a> elements within the <td>
        a_elements = td.find_all('a')
        # Print the text and href attribute of each <a> element
        for a in a_elements:
            Unis["Universidade"].append(a.text)
            Unis["Link_DGES"].append(a["href"])
    time.sleep(1)
    
i=0

col_ids=[("Estado da IES","iuo_eiuo_id"),("Subsistema","iuo_s_id"),("Natureza","iuo_n_id"),("Tipo de ensino","iuo_ti_id"),("Link_Website","iuo_link"),("Morada","iuo_morada"),
           ("Telefone","iuo_telefone"),("Código Postal","iuo_cod_postal"),("Email","iuo_email"),("Distrito","iuo_link")]

for link in Unis["Link_DGES"]:
    browser.get(link)
    html = browser.page_source
    soup = BeautifulSoup(html, 'html.parser') # could be BeautifulSoup(html_content, 'lxml')?
    dom = etree.HTML(str(soup)) 
    Unis["Nome"].append(dom.xpath('/html[1]/body[1]/div[4]/div[1]/div[1]/div[1]/div[1]/h1[1]')[0].text)
    try: # check if unidade principal
        Unis["Unidade Organica"].append(dom.xpath("/html[1]/body[1]/div[4]/div[1]/div[1]/div[1]/div[1]/h2[1]")[0].text)
    except:
        Unis["Unidade Organica"].append("Unidade Principal")
    #    
    for tuple in col_ids:
        try:
            Unis[tuple[0]].append(soup.find('input', id=tuple[1])['value']) 
        except:
            Unis[tuple[0]].append("Sem "+ tuple[0]) 
    i+=1

IndexError: list index out of range

In [ ]:
df_Unis = pd.DataFrame(Unis)
df_Unis
df_Unis.to_csv('Unis.csv', index=False)

,Universidade,Link_DGES
0,Escola Superior de Enfermagem de Coimbra,https://www.dges.gov.pt/simges/public/www/deta...
1,Escola Superior de Enfermagem de Lisboa,https://www.dges.gov.pt/simges/public/www/deta...
2,Escola Superior de Enfermagem do Porto,https://www.dges.gov.pt/simges/public/www/deta...
3,Escola Superior de Hotelaria e Turismo do Estoril,https://www.dges.gov.pt/simges/public/www/deta...
4,Escola Superior Náutica Infante D. Henrique,https://www.dges.gov.pt/simges/public/www/deta...
...,...,...
1642,Unidade Politécnica Militar - Instituto Univer...,https://www.dges.gov.pt/simges/public/www/deta...
1643,Academia Militar - Instituto Universitário Mil...,https://www.dges.gov.pt/simges/public/www/deta...
1644,Academia da Força Aérea - Instituto Universitá...,https://www.dges.gov.pt/simges/public/www/deta...
1645,Escola Naval - Instituto Universitário Militar,https://www.dges.gov.pt/simges/public/www/deta...
